# Combine All ATP Synthase and Proton Pump Models
---

In [2]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

from subsbml import createNewSubsystem, createSubsystem

Loading BokehJS ...

In [9]:
palette = colorcet.b_glasbey_category10
palette.remove( '#d62628')

In [3]:
# atp synthase models
ss1_atpsyn_txtl = createSubsystem('atpsynthase_tx_tl.xml', subsystemName = 'atpsyn_txtl')
ss2_atpsyn_mi = createSubsystem('atpsynthase_membrane_integration.xml', subsystemName = 'atpsyn_mi')
ss3_atpsynthesis = createSubsystem('atp_synthesis_through_atpsynthase.xml', subsystemName = 'atpsynthesis')

# proton pump ss
ss4_pp_txtl = createSubsystem('proton_pump_tx_tl.xml', subsystemName = 'pp_txtl')
ss5_pp_mi = createSubsystem('proton_pump_membrane_integration.xml', subsystemName = 'pp_mi')
ss6_protontransport = createSubsystem('proton_transport_through_proton_pump.xml', subsystemName = 'protontransport')


In [4]:
atp_subsystem = ss1_atpsyn_txtl + ss2_atpsyn_mi + ss3_atpsynthesis

In [6]:
spec_list_10 = ['metabolite_atp', 'proton_outside_H', 'dna_atpsyn', 'protein_RNAP_machinery',
               'protein_Ribo_machinery', 'protein_RNAase_machinery']
for i,spec in enumerate(spec_list_10):
    atp_subsystem.setSpeciesAmount(spec,10)


In [23]:
timepoints = np.linspace(0,10,100)
results, _ = atp_subsystem.simulateWithBioscrape(timepoints, sbml_warnings = False)

In [24]:
ATP_id = atp_subsystem.getSpeciesByName('metabolite_atp').getId()
ADP_id = atp_subsystem.getSpeciesByName('metabolite_adp').getId()

outside_H_id = atp_subsystem.getSpeciesByName('proton_outside_H').getId()
inside_H_id = atp_subsystem.getSpeciesByName('proton_inside_H').getId()
atp_synthase_protein_id = atp_subsystem.getSpeciesByName('protein_atpsyn').getId()

In [25]:
p = bokeh.plotting.figure(width = 450, height = 300)
plot_list = [ATP_id, outside_H_id, inside_H_id, atp_synthase_protein_id]

for i,item in enumerate(plot_list):
    p.line(timepoints, results[item], line_width = 2, legend_label = str(item), color = palette[i])
    
bokeh.io.show(p)
